# Hidden Markov Model for Part-of-Speech Tagging

This notebook implements a Hidden Markov Model (HMM) from scratch to perform part-of-speech (POS) tagging using the Brown corpus with the universal tagset. POS tagging is a classic sequence labeling task in NLP where each word in a sentence is assigned its grammatical category (e.g., noun, verb, adjective).

We walk through the following steps:

- **Data Preparation**: Load and split the Brown corpus into training and test sets.
- **Frequency Counts**: Compute transition and emission counts needed for estimating probabilities.
- **Probability Estimation**: Calculate initial, transition, and emission probabilities using maximum likelihood estimates.
- **Viterbi Algorithm**: Implement the Viterbi algorithm to infer the most likely sequence of POS tags for a sentence.
- **Evaluation**: Assess tagging accuracy on the test set.

This notebook offers a hands-on understanding of how probabilistic sequence models like HMMs work under the hood, before diving into neural sequence models. This is part of **Week 2 - Classical NLP Techniques** in my learning journey.

## Import Libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import brown
from collections import defaultdict

from collections import Counter

import math

In [2]:
# download necessary nltk packages that we will use as our dataset
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

## Loading and Splitting the Data

We begin by loading the [Brown Corpus](https://www.nltk.org/nltk_data/) using the `universal` tagset, which provides a simplified set of part-of-speech (POS) tags. This tagset groups POS labels into categories like `NOUN`, `VERB`, `ADJ`, etc., making it easier to interpret results.

The dataset is then split into training and testing sets using an 80/20 split. The training data will be used to estimate the probabilities required by the Hidden Markov Model (HMM), while the test data will be used to evaluate the model’s tagging performance.

In [3]:
tagged_sentences = brown.tagged_sents(tagset = 'universal')

In [4]:
train_size = int(len(tagged_sentences) * 0.8)
train_sentences = tagged_sentences[:train_size]
test_sentences = tagged_sentences[train_size:]

In [5]:
print('Length of train sentences:', len(train_sentences))
print('Length of test senetences:', len(test_sentences))

Length of train sentences: 45872
Length of test senetences: 11468


### Vocabulary and Tag Distribution

Next, we extract the words and tags from the training sentences:

- **Words** are lowercased to reduce sparsity in the vocabulary.
- **Tags** are collected from the part-of-speech annotations.

We then compute:
- The set of unique words and tags,
- The frequency of each word and tag using Python’s `Counter`.

This gives us insight into the size of our vocabulary and the distribution of POS tags, which will be useful for initializing and smoothing our Hidden Markov Model. We also print the 10 most common words and a full distribution of tag frequencies.

In [6]:
words = [word.lower() for sent in train_sentences for word, _ in sent]
tags = [tag for sent in train_sentences for _, tag in sent]

In [7]:
unique_words = set(words)
unique_tags = set(tags)

In [8]:
word_counts = Counter(words)
tag_counts = Counter(tags)

In [9]:
print('Number of unique words:', len(unique_words))
print('Number of unique tags:', len(unique_tags))

Number of unique words: 45755
Number of unique tags: 12


In [10]:
print('Most common words:', word_counts.most_common(10))
print('Most common tags:', tag_counts)

Most common words: [('the', 61191), (',', 48492), ('.', 39534), ('of', 32942), ('and', 24281), ('to', 22423), ('a', 19499), ('in', 18935), ('is', 9671), ('that', 9009)]
Most common tags: Counter({'NOUN': 241528, 'VERB': 150459, 'ADP': 126332, '.': 118482, 'DET': 116989, 'ADJ': 73866, 'ADV': 45940, 'PRON': 35550, 'CONJ': 32177, 'PRT': 23316, 'NUM': 13802, 'X': 1205})


## Constructing HMM Count Matrices

We now build the count matrices required for a Hidden Markov Model (HMM):

- **Emission Counts**: How often each word is observed given a tag (P(word | tag)).
- **Transition Counts**: How often each tag follows another tag (P(tag_t | tag_{t-1})).
- **Initial Tag Counts**: Frequency of tags that appear at the beginning of a sentence.

This loop goes through each sentence in the training data and:
- Increments the emission count for the observed `(tag, word)` pair.
- Tracks initial tag counts for the first tag in every sentence.
- Updates transition counts between consecutive tags.

These frequency counts will later be normalized to form probability matrices for decoding with the Viterbi algorithm.

In [11]:
transition_counts = defaultdict(lambda: defaultdict(int))
emission_counts = defaultdict(lambda: defaultdict(int))
tag_counts = dict(tag_counts)
initial_counts = defaultdict(int)

In [12]:
for sentence in train_sentences:
  prev_tag = None
  for i, (word, tag) in enumerate(sentence):
    word = word.lower()
    emission_counts[tag][word] += 1

    if i == 0:
      initial_counts[tag] += 1
    else:
      transition_counts[prev_tag][tag] += 1

    prev_tag = tag

## Converting Counts to Probabilities

Once we’ve collected the raw frequency counts for transitions, emissions, and initial tags, we convert them into probabilities.

- **Transition Probabilities**: Normalize each row of the `transition_counts` dictionary so that each value represents P(tag_t | tag_{t−1}).
- **Emission Probabilities**: Normalize each row of the `emission_counts` dictionary to represent P(word | tag).
- **Initial Probabilities**: Normalize the tag frequencies that appear at the start of each sentence to get P(tag_0).

This step turns the HMM components into valid probability distributions, which we’ll use to perform POS tagging on unseen sentences.

In [13]:
def convert_counts_to_probabilities(counts):
  """
  Convert nested count dictionaries to probability distributions.

  Parameters:
      counts (dict): A nested dictionary where each outer key maps to a dictionary
                      of counts (e.g., {state: {next_state: count, ...}}).

  Returns:
      dict: A nested dictionary with the same structure, but with normalized
            probabilities instead of raw counts.
            (e.g., {state: {next_state: prob, ...}})
  """
  probs = {}
  for key, counts in counts.items():
    total = sum(counts.values())
    probs[key] = {k: v / total for k, v in counts.items()}
  return probs

In [14]:
transition_probabilities = convert_counts_to_probabilities(transition_counts)
emission_probabilities = convert_counts_to_probabilities(emission_counts)
initial_probabilities = {k: v / sum(initial_counts.values()) for k, v in initial_counts.items()}

## Emission Probability Function

This function retrieves the probability of a word given a specific tag. If the word was not seen with the tag during training, it returns a small smoothing value to prevent zero probability.

In [15]:
def get_emission_probability(emission_probs, tag, word, smoothing = 1e-6):
  """
  Retrieve the emission probability of a word given a tag, with optional smoothing.

  Args:
      emission_probs (dict): A nested dictionary of emission probabilities,
                              where emission_probs[tag][word] gives P(word | tag).
      tag (str): The POS tag or state.
      word (str): The observed word/token.
      smoothing (float, optional): A small fallback probability used if the word
                                    is not found under the given tag. Default is 1e-6.

  Returns:
      float: The emission probability P(word | tag), or the smoothing value if the word is unseen.

  Notes:
      - This function uses add-ε smoothing to handle unseen word-tag pairs.
      - If the word is not found in the emission dictionary for the given tag,
        the function returns the smoothing value instead.
  """
  return emission_probs[tag].get(word, smoothing)

## Viterbi Algorithm for POS Tagging

This function implements the Viterbi algorithm to find the most probable sequence of tags for a given sentence.

It uses:
- **Initial probabilities** for the first word.
- **Transition probabilities** between tags.
- **Emission probabilities** of a word given a tag.

The algorithm builds a dynamic programming table (`v_table`) and a backpointer matrix to trace the best tag sequence. The output is a list of (word, tag) pairs for the sentence along with the overall path probability.

In [18]:
def viterbi(sentence, initial_probs, transition_probs, emission_probs, unique_tags):
  """
  Apply the Viterbi algorithm to find the most likely sequence of tags for a given sentence.

  Parameters:
      sentence (list of str): A list of words representing the observed sequence.
      initial_probs (dict): A dictionary of initial probabilities for each tag (P(tag_0)).
      transition_probs (dict): A nested dictionary representing transition probabilities
                                between tags (P(tag_t | tag_{t-1})).
      emission_probs (function): A function that returns the emission probability
                                  P(word | tag), typically with smoothing support.
      unique_tags (set or list of str): The set of all possible tags used in the model.

  Returns:
      tuple:
          - list of (word, tag) pairs representing the most likely tag sequence.
          - float: the probability of the best tag sequence.

  Notes:
      - Uses log probabilities to avoid numerical underflow during multiplication.
      - Applies add-one smoothing (1e-6) for unseen emission and transition probabilities.
      - Backpointers are used to reconstruct the best tag sequence from the dynamic programming table.
  """
  words = [word.lower() for word in sentence]
  n = len(sentence)
  m = len(unique_tags)
  tags_list = list(unique_tags)

  v_table = np.zeros((m, n))
  backpointer = np.zeros((m, n), dtype = int)

  for i, tag in enumerate(tags_list):
    v_table[i, 0] = math.log(initial_probs.get(tag, 1e-6)) + math.log(get_emission_probability(emission_probs, tag, words[0]))

  for i in range(1, n):
    for j in range(m):
      temp_values = []
      for k in range(m):
        temp_values.append(
            v_table[k, i - 1] + math.log(transition_probs.get(tags_list[k], {}).get(tags_list[j], 1e-6)) + math.log(get_emission_probability(emission_probs, tags_list[j], words[i]))
        )
      v_table[j, i] = max(temp_values)
      backpointer[j, i] = np.argmax(temp_values)

  best_path_prob = max(v_table[:, -1])
  best_path_pointer = np.argmax(v_table[:, -1])

  best_path = [best_path_pointer]
  for i in range(n - 1, 0, -1):
    best_path.append(int(backpointer[best_path[-1], i]))

  best_tags = [tags_list[i] for i in best_path[::-1]]

  return list(zip(sentence, best_tags)), math.exp(best_path_prob)

## Evaluating the POS Tagger

This function evaluates the overall tagging accuracy of the Viterbi decoder on the test set. It compares predicted tags to ground truth tags and reports the proportion of correct predictions.

In [19]:
def evaluate(test_data, initial_probs, transition_probs, emission_probs, unique_tags):
  """
  Evaluate the accuracy of a sequence tagging model using the Viterbi algorithm.

  Parameters:
      test_data (list of list of (str, str)): A list of sentences, where each sentence is a
                                              list of (word, true_tag) pairs.
      initial_probs (dict): Initial tag probabilities P(tag_0).
      transition_probs (dict): Transition probabilities P(tag_t | tag_{t-1}).
      emission_probs (function): A function that returns emission probabilities P(word | tag).
      unique_tags (set or list): The full set of possible tags.

  Returns:
      float: The overall tagging accuracy (correct predictions / total tags).

  Notes:
      - Uses the Viterbi decoder to generate predicted tag sequences.
      - Assumes test data uses the same tag set and vocabulary as the training data.
  """
  correct, total = 0, 0
  for sentence in test_data:
    words, true_tags = zip(*sentence)

    predictions, probability = viterbi(words, initial_probs, transition_probs, emission_probs, unique_tags)
    predicted_tags = [tag for _, tag in predictions]

    correct += sum(pred == true for pred, true in zip(predicted_tags, true_tags))
    total += len(true_tags)

  return correct / total

## Results

After running the Viterbi algorithm on the test set, we compute the overall tagging accuracy. This reflects how well our HMM-based POS tagger generalizes to unseen data.

In [21]:
accuracy = evaluate(test_sentences, initial_probabilities, transition_probabilities, emission_probabilities, unique_tags)
print(f'Accuracy: {accuracy*100:.4f}%')

Accuracy: 94.6014%
